In [1]:
# running as docker?
docker_running = False

# define repo path and add it to the path
from pathlib import Path
import sys, os
if not docker_running: # if we are running locally
    repo_path= Path.cwd().resolve()
    while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
        repo_path = repo_path.parent #go up one level
else: # if running in the container
    repo_path = Path('opt/usuari')
sys.path.insert(0,str(repo_path)) if str(repo_path) not in sys.path else None

In [5]:
folder = repo_path / 'data/challange_2023/Val/DATA'
image_paths = list(folder.glob("*"))
# order by name
image_paths.sort()
image_paths

[PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_100.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_101.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_102.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_103.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_104.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_105.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_106.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_107.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2023/Val/DATA/DATA_108.nrrd'),
 PosixPath('/home/ricardo/ABUS2023_documents/tdsc_abus23/data/challange_2

In [2]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
from tqdm import tqdm

In [14]:
def get_center_length(mask):

    # Get the non-zero indices
    non_zero_indices = np.nonzero(mask)

    min_z = np.min(non_zero_indices[2])
    min_y = np.min(non_zero_indices[1])
    min_x = np.min(non_zero_indices[0])
    max_z = np.max(non_zero_indices[2])
    max_y = np.max(non_zero_indices[1])
    max_x = np.max(non_zero_indices[0])

    # Get the center of the lesion
    center_z = (min_z + max_z) / 2
    center_y = (min_y + max_y) / 2
    center_x = (min_x + max_x) / 2

    center = [center_x, center_y, center_z]

    # Get the length of the lesion
    length_z = max_z - min_z
    length_y = max_y - min_y
    length_x = max_x - min_x

    length = [length_x, length_y, length_z]

    return center, length

In [30]:
file = repo_path / 'predict/Segmentation/MASK_100.nrrd'

# read the image
mask = sitk.ReadImage(str(file))
mask = sitk.GetArrayFromImage(mask)

# get the center and length of the lesion
center, length = get_center_length(mask)

# compute mean of probabilities inside the lesion
probs_path = repo_path / 'cached_data/probs/prob_map.npy'
probs = np.load(probs_path)

inner_probs = probs[mask==1]
mean_inner_probs = np.mean(inner_probs)

dataframe = pd.DataFrame(columns=['public_id', 'coordX', 'coordY', 'coordZ', 'x_length', 'y_length', 'z_length', 'probability'])
dataframe.loc[0] = [file.stem.split('_')[1], center[0], center[1], center[2], length[0], length[1], length[2], mean_inner_probs]
# append to the main dataframe
# main_dataframe = pd.concat([main_dataframe, dataframe], ignore_index=True)

dataframe

,public_id,coordX,coordY,coordZ,x_length,y_length,z_length,probability
0,100,201.0,243.0,323.0,40,360,276,0.968262
